In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/733k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/6.36M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/657k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [3]:
print(dataset)

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})


In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [5]:
tokenized_dataset = dataset.map(
    lambda example : tokenizer(example['text']),
    batched = True
)
tokenized_dataset = tokenized_dataset.remove_columns(['text'])
print(tokenized_dataset)

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 3760
    })
})


In [6]:
block_size = 128

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = (len(concatenated["input_ids"]) // block_size) * block_size
    result = {
        k: [t[i: i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [7]:
print(lm_dataset)

DatasetDict({
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2210
    })
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 18667
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1930
    })
})


In [8]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [9]:
from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling

training_args = TrainingArguments(
    output_dir="./gpt2-finetuned-wikitext",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_dir="./logs",
    report_to="none",
    load_best_model_at_end=True,
)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)
trainer.train()


/tmp/ipython-input-9-2805276175.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.466600,3.411082
2,3.336700,3.394568
3,3.288500,3.393095


There were missing keys in the checkpoint model loaded: ['lm_head.weight'].


TrainOutput(global_step=7002, training_loss=3.3865214611795484, metrics={'train_runtime': 2282.2064, 'train_samples_per_second': 24.538, 'train_steps_per_second': 3.068, 'total_flos': 3658153771008000.0, 'train_loss': 3.3865214611795484, 'epoch': 3.0})

In [10]:
import math

eval_results = trainer.evaluate(lm_dataset['test'])
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")


Perplexity: 29.58


In [12]:
import torch

def compute_top_k_accuracy(model, tokenizer, dataset, k=5):
    model.eval()
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for example in dataset:
        inputs = torch.tensor([example["input_ids"][:-1]]).to(device)
        labels = torch.tensor(example["input_ids"][1:]).to(device)
        with torch.no_grad():
            outputs = model(inputs)
            logits = outputs.logits[0, -1]  # last token logits
            top_k = torch.topk(logits, k).indices
            if labels[-1] in top_k:
                correct += 1
            total += 1
    return correct / total

top_k_acc = compute_top_k_accuracy(model, tokenizer, lm_dataset["validation"])
print(f"Top-5 Accuracy: {top_k_acc:.2%}")

Top-5 Accuracy: 63.83%


In [18]:
# Save final artifacts
trainer.save_model("my-finetuned-gpt2")
tokenizer.save_pretrained("my-finetuned-gpt2")

# Push to Hub
from transformers import AutoModelForCausalLM, AutoTokenizer

model = AutoModelForCausalLM.from_pretrained("my-finetuned-gpt2")
tokenizer = AutoTokenizer.from_pretrained("my-finetuned-gpt2")

model.push_to_hub("koushik-25/my-finetuned-gpt2")
tokenizer.push_to_hub("koushik-25/my-finetuned-gpt2")


Uploading...:   0%|          | 0.00/498M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/koushik-25/my-finetuned-gpt2/commit/6d5aa1966d5c0634cfdc25de657baa03356b9a5c', commit_message='Upload tokenizer', commit_description='', oid='6d5aa1966d5c0634cfdc25de657baa03356b9a5c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/koushik-25/my-finetuned-gpt2', endpoint='https://huggingface.co', repo_type='model', repo_id='koushik-25/my-finetuned-gpt2'), pr_revision=None, pr_num=None)

In [25]:
from transformers import pipeline

generator = pipeline("text-generation", model="koushik-25/my-finetuned-gpt2")


Device set to use cpu


In [35]:
def text_generator(generator, max_length, input_text) :
    output = generator(input_text, max_length = max_length)
    return output[0]['generated_text']

In [37]:
input_text = "Once upon a time,"
output = text_generator(generator, 30, input_text)
print(output)

Both `max_new_tokens` (=256) and `max_length`(=30) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Once upon a time, it was believed that a malevolent force was behind the destruction of the planet. This theory was accepted by the survivors of the first human settlement on Venus in the late 23rd century, and the first documented instances of such a presence on the planet. The discovery of a large asteroid belt on the Moon was also accepted by the scientific community as an example of a massive transit of the Earth's crust. The first recorded sightings of a large asteroid occurred on June 23, 1973, when the space rock was discovered ; it was believed that the asteroid had struck the moon before, and that it was a small asteroid. The discovery of a large asteroid belt on the Moon was further confirmed by the discovery of a rock on the moon. 
 In the early 21st century, scientists have proposed that the asteroid belt originated from a collision with a star that was nearby in the outer Solar System. This theory has been supported by the results of observations showing that the asteroid 